# Σημειωματάριο τρίτο: Matching the DataFrames

## Όπου ενώνουμε τα tweets με τα CVE-IDs στα οποία αναφέρονται

Έχοντας σχηματίσει τα σύνολα δεδομένων με τα tweets και τις εγγραφές των ευπαθειών, σε αυτό το notebook αντιστοιχίζουμε κάθε tweet με τις ευπάθειες στις οποίες αυτό αναφέρεται. Ο **σκοπός** μας είναι να δημιουργήσουμε ένα dataset με tweets που αναφέρονται σε έγκυρους κωδικούς CVE-IDs, έτσι ώστε να αναλύσουμε σε επόμενα notebooks την δημοτικότητα των ευπαθειών.

In [1]:
import pandas as pd
import numpy as np
import re 

In [2]:
# Χρησιμοποιούμε την παράμετρο lineterminator γιατί χωρίς αυτήν τα δεδομένα δεν "διαβάζονται" ορθά. 
# Ο λόγος είναι ίσως ότι τα CSV αρχεία δημιουργήθηκαν σε υπολογιστή με λειτουργικό σύστημα Linux.
tweets = pd.read_csv('Data/tweets/tweets_2021_with_users.csv', lineterminator='\n')
records = pd.read_csv('Data/NVD/NVD_records_2022_downoload_at_February_2022_01-02-2022.csv', lineterminator='\n')

Στην συνέχεια περιγράφεται ο αλγόριθμος που υλοποιεί το matching. 

Για κάθε ένα από τα tweets εκτελούνται τα ακόλουθα βήματα:
1. Χρησιμοποιώντας regular expression, εντοπίζουμε αρχικά όλα τα CVE IDs που υπάρχουν στο κείμενο του tweet.
2. Στην συνέχεια, για κάθε ένα από αυτά τα IDs που βρήκαμε στο προηγούμενο βήμα, κρατάμε σε μια λίστα τα έγκυρα IDs, δηλαδή τα IDs που υπάρχουν ως εγγραφές στο NVD DataFrame μας το οποίο περιέχει τις εγγραφές με τα επίσημα δεδομένα ευπαθειών από την NVD.
3. Προσθέτουμε στην λίστα data τα έγκυρα CVE IDs που βρήκαμε σε αυτό το tweet. Η λίστα αυτή περιέχει για κάθε tweet μια λίστα με τα έγκυρα CVE IDs που αναφέρονται σε αυτό ή NaN εάν στο συγκεκριμένο tweet δεν αναφέρεται κανένα έγκυρο CVE ID.

Η λίστα data που είναι η έξοδος της εκτέλεσης του παραπάνω αλγόριθμου, είναι παράλληλη λίστα με την texts, η οποία περιέχει το κείμενο του κάθε tweet.

Θυμίζουμε ότι δεν υπάρχουν CVE IDs για ευπάθειες που είναι RESERVED, REJECT, DISPUTED, UNVERIFIABLE και PRERELEASE.
Οπότε εάν ο αλγόριθμος συναντίσει στο κείμενο ενός tweet τέτοιου είδους CVE IDs δεν τα θεωρεί έγκυρα.

In [5]:
# Κρατάμε στην λίστα texts την στήλη με τα κείμενα των tweets και στην λίστα nvd_ids τα CVE IDs από την NVD.
texts = list(tweets['text'])  
nvd_ids = list(records['cve_id'])
data = [] 

In [7]:
# Χρησιμοποιούμε έναν μετρητή για να εμφανίζουμε ανά 10.000 τα tweets που έχουμε προσπελάσει.
i = 0 

for text in texts:
    if (i % 10000 == 0):
        print("Έως τώρα έχουν προσπελαστεί", i, "tweets.")
    i = i +1   
    
    try:
        
        # Η valid_matched_ids είναι η υπολίστα στην οποία αποθηκεύονται τα CVE IDs που αναφέρονται στο κείμενο
        # του συγκεκριμένου tweet και έχουν ελεγχθεί ότι περιλαμβάνονται στην NVD.
        valid_matched_ids = []      
        
        # Με την findall βρίσκουμε τα CVE IDs που αναφέρονται στο tweet χρησιμοποιώντας regular expression.
        # Είναι όμως απαραίτητος ο έλεγχος που 
        # γίνεται στην συνέχεια έτσι ώστε να μην κρατήσουμε μη έγκυρα CVE IDs (δηλ. IDs που δεν υπάρχουν στο
        # dataset που φτιάξαμε από τα επίσημα δεδομένα της NVD). Η findall επιστρέφει μια λίστα με τα 
        # αλφαριθμητικά που ταιριάζουν στο pattern.
        #
        # Η findall επιστρέφει μια λίστα με τα strings που ταιριάζουν στο pattern που έχουμε δηλώσει, και αυτό σημαίνει
        # ότι επιτρέπει τα διπλότυπα. Έτσι εάν σε ένα tweet έχουμε ένα ID που αναφέρεται δύο φορές, θα αποθηκευτεί
        # στη λίστα δύο φορές. Εμείς όμως θέλουμε από κάθε tweet να κρατάμε τα μοναδικά IDs που αναφέρονται σε αυτό.
        # Οπότε μετατρέπουμε την λίστα σε set κι έτσι διαγράφουμε τα διπλότυπα.
        matched_ids = set(re.findall("CVE-\d{1,6}-\d{1,6}", text))  
        
        for matched in matched_ids:
            if matched in nvd_ids:
                valid_matched_ids.append(matched)
                
        # Εάν δεν βρήκαμε κανένα έγκυρο CVE ID στο κείμενο του text τότε καταχώρησε np.NaN
        # στην θέση της λίστας data που αντιστοιχεί στο tweet αυτό.        
        if (len(valid_matched_ids) == 0):
            data.append(np.NaN)     
   
        else:
            # Αποθήκευση των έγκυρων IDs που εντοπίσαμε στο κείμενο.
            data.append(valid_matched_ids)        
        
    except Exception as e:
        print("Ένα σφάλμα εμφανίστηκε:")
        print(e)
        break

Έως τώρα έχουν προσπελαστεί 0 tweets.
Έως τώρα έχουν προσπελαστεί 10000 tweets.
Έως τώρα έχουν προσπελαστεί 20000 tweets.
Έως τώρα έχουν προσπελαστεί 30000 tweets.
Έως τώρα έχουν προσπελαστεί 40000 tweets.
Έως τώρα έχουν προσπελαστεί 50000 tweets.
Έως τώρα έχουν προσπελαστεί 60000 tweets.
Έως τώρα έχουν προσπελαστεί 70000 tweets.
Έως τώρα έχουν προσπελαστεί 80000 tweets.
Έως τώρα έχουν προσπελαστεί 90000 tweets.
Έως τώρα έχουν προσπελαστεί 100000 tweets.
Έως τώρα έχουν προσπελαστεί 110000 tweets.
Έως τώρα έχουν προσπελαστεί 120000 tweets.
Έως τώρα έχουν προσπελαστεί 130000 tweets.
Έως τώρα έχουν προσπελαστεί 140000 tweets.
Έως τώρα έχουν προσπελαστεί 150000 tweets.
Έως τώρα έχουν προσπελαστεί 160000 tweets.
Έως τώρα έχουν προσπελαστεί 170000 tweets.
Έως τώρα έχουν προσπελαστεί 180000 tweets.
Έως τώρα έχουν προσπελαστεί 190000 tweets.
Έως τώρα έχουν προσπελαστεί 200000 tweets.
Έως τώρα έχουν προσπελαστεί 210000 tweets.


Εφόσον η αντιστοίχηση των tweets με τα CVE IDs ολοκληρώθηκε επιτυχώς, εισάγουμε την λίστα **data** που περιέχει τις αντιστοιχίες, σαν νέα στήλη στο DataFrame **tweets**.

In [8]:
if (len(tweets) == len(data)):
    print("Το matching ολοκληρώθηκε επιτυχώς! Η λίστα data προστίθεται σαν νέα στήλη στο DataFrame tweets.")
    tweets.insert(2, 'matched_CVE_IDs', data)
else:
    print("Κάτι πήγε στραβά και το μέγεθος της λίστας data δεν είναι ίσο με\n το μέγεθος του dataframe tweets οπότε δεν μπορούμε να τα ενώσουμε...Ξαναδές το!")

Το matching ολοκληρώθηκε επιτυχώς! Η λίστα data προστίθεται σαν νέα στήλη στο DataFrame tweets.


In [9]:
tweets

id  \
0       1458428897368940553   
1       1458428424326983680   
2       1458428142373179392   
3       1458427987301371911   
4       1458427985753776138   
...                     ...   
210094  1346143661831172097   
210095  1346143642998743043   
210096  1346143619967811586   
210097  1346143576116375555   
210098  1346143574568611840   

                                                     text   matched_CVE_IDs  \
0       CVE-2021-34582\n\nIn Phoenix Contact FL MGUARD...  [CVE-2021-34582]   
1       SAP объявила об исправлении критических уязвим...               NaN   
2       CVE-2021-34598\n\nIn Phoenix Contact FL MGUARD...  [CVE-2021-34598]   
3       New post from https://t.co/uXvPWJy6tj (CVE-202...  [CVE-2021-39474]   
4       New post from https://t.co/9KYxtdZjkl (CVE-202...  [CVE-2021-39474]   
...                                                   ...               ...   
210094  CVE-2021-26287 is called Confident Sickle\nhtt...               NaN   
210095  CVE-2021-26286 shall henceforth be named Relia...               NaN   
210096  One night, CVE-2021-25847 wished upon a star, ...  [CVE-2021-25847]   
210097  Let the annals of the day show that CVE-2021-1...               NaN   
210098  My real name is CVE-2021-16268 but all my frie...               NaN   

                      created_at  retweet_count  reply_count  like_count  \
0       2021-11-10T13:38:40.000Z              0            0           0   
1       2021-11-10T13:36:47.000Z              0            0           0   
2       2021-11-10T13:35:40.000Z              0            0           0   
3       2021-11-10T13:35:03.000Z              0            0           0   
4       2021-11-10T13:35:02.000Z              0            0           0   
...                          ...            ...          ...         ...   
210094  2021-01-04T17:17:13.000Z              0            0           0   
210095  2021-01-04T17:17:09.000Z              0            0           0   
210096  2021-01-04T17:17:03.000Z              0            0           0   
210097  2021-01-04T17:16:53.000Z              0            0           0   
210098  2021-01-04T17:16:52.000Z              0            0           0   

        quote_count            author_id                author_name  \
0                 0   941389496771399680  Vulmon Vulnerability Feed   
1                 0  1132127578922344448            IT news for all   
2                 0   941389496771399680  Vulmon Vulnerability Feed   
3                 0   955014888446939136             Wolfgang Sesin   
4                 0   958005194398289920               www.sesin.at   
...             ...                  ...                        ...   
210094            0  1096056138569842688                   vulnonym   
210095            0  1096056138569842688                   vulnonym   
210096            0  1096056138569842688                   vulnonym   
210097            0  1096056138569842688                   vulnonym   
210098            0  1096056138569842688                   vulnonym   

        author_username                                 author_description  \
0           VulmonFeeds  Vulnerability Feed Bot (tweets new and some ol...   
1       IT_news_for_all  Сбор новостей с каналов ИТ тематики\n🔥🎯https:/...   
2           VulmonFeeds  Vulnerability Feed Bot (tweets new and some ol...   
3         WolfgangSesin  Check Point Master & Instructor, Pentest Exper...   
4          www_sesin_at  for more information about us please visit htt...   
...                 ...                                                ...   
210094         vulnonym            I'm a bot generating names for CVE IDs.   
210095         vulnonym            I'm a bot generating names for CVE IDs.   
210096         vulnonym            I'm a bot generating names for CVE IDs.   
210097         vulnonym            I'm a bot generating names for CVE IDs.   
210098         vulnonym            I'm a bot generating names for CVE IDs.   



Θα πρέπει στη συνέχεια να διαχειριστούμε τα tweets που δεν αναφέρονται σε έγκυρα CVE IDs.

In [10]:
if (data.count(np.NaN) == 0):
    print("Τέλεια, κάθε tweet στο dataset αναφέρεται σε τουλάχιστον ένα έγκυρο CVE ID!")
else:
    print("Έχουμε ",data.count(np.NaN), " tweets τα οποία δεν αναφέρονται σε κάποιο έγκυρο CVE ID.")

Έχουμε  10484  tweets τα οποία δεν αναφέρονται σε κάποιο έγκυρο CVE ID.


In [11]:
tweets.isna().sum()

id                            0
text                          0
matched_CVE_IDs           10484
created_at                    0
retweet_count                 0
reply_count                   0
like_count                    0
quote_count                   0
author_id                     0
author_name                   6
author_username               0
author_description         3863
author_followers_count        0
author_following_count        0
author_tweet_count            0
author_listed_count           0
dtype: int64

Παρατηρούμε ότι 10.484 tweets από το σύνολο των 210.099 δεν αναφέρονται σε κάποιο έγκυρο CVE ID.
Θα πρέπει να διαγραφούν από το dataset πριν προχωρήσουμε στην ανάλυση των δεδομένων.

In [12]:
tweets = tweets.dropna(subset=['matched_CVE_IDs'])

In [13]:
tweets.isna().sum()

id                           0
text                         0
matched_CVE_IDs              0
created_at                   0
retweet_count                0
reply_count                  0
like_count                   0
quote_count                  0
author_id                    0
author_name                  5
author_username              0
author_description        3182
author_followers_count       0
author_following_count       0
author_tweet_count           0
author_listed_count          0
dtype: int64

Τα tweets που δεν αναφέρονται σε έγκυρο CVE ID έχουν πια διαγραφεί.

Τα missing values στην στήλη 'author_description' ήταν αναμενόμενα. Όχι όμως αυτά της στήλης 'author_name', αφού στο πρώτο notebook όπου συλλέγουμε τα tweets δεν φαίνονταν να υπάρχουν ελλιπείς τιμές σε αυτό το χαρακτηριστικό πριν δημιουργήσουμε το αρχείο CSV. Ας δούμε τί βρίσκεται εκεί...

In [14]:
tweets.loc[tweets['author_name'].isna()]

id  \
4748    1456644114124926979   
8027    1455595683994476548   
71760   1428376802393792516   
105764  1412421816400986122   
173827  1374023488617336842   

                                                     text   matched_CVE_IDs  \
4748    A heap overflow #vulnerability (CVE-2021-43267...  [CVE-2021-43267]   
8027    WARNING: A critical unauthenticated remote cod...  [CVE-2021-22205]   
71760   FireEye has disclosed a new critical #vulnerab...  [CVE-2021-28372]   
105764  #Microsoft is urging #Azure users to update th...  [CVE-2021-26701]   
173827  A critical #vulnerability (CVE-2021-22986 / CV...  [CVE-2021-22986]   

                      created_at  retweet_count  reply_count  like_count  \
4748    2021-11-05T15:26:34.000Z              2            0           5   
8027    2021-11-02T18:00:29.000Z              2            0           2   
71760   2021-08-19T15:22:22.000Z              1            0           2   
105764  2021-07-06T14:42:57.000Z              2            0           2   
173827  2021-03-22T15:41:42.000Z              2            0           1   

        quote_count           author_id author_name author_username  \
4748              0  717369822007275520         NaN           rul3r   
8027              0  717369822007275520         NaN           rul3r   
71760             1  717369822007275520         NaN           rul3r   
105764            0  717369822007275520         NaN           rul3r   
173827            0  717369822007275520         NaN           rul3r   

       author_description  author_followers_count  author_following_count  \
4748                  NaN                    1798                       0   
8027                  NaN                    1798                       0   
71760                 NaN                    1798                       0   
105764                NaN                    1798                       0   
173827                NaN                    1798                       0   

        author_tweet_count  author_listed_count  
4748                  4691                  111  
8027                  4691                  111  
71760                 4691                  111  
105764                4691                  111  
173827                4691                  111

Συμπαιραίνουμε ότι εδώ η βιβλιοθήκη pandas θεωρεί missing values τις τιμές της στήλη 'author_name' για έναν συγκεκριμένο λογαριασμό επειδή έχει για όνομα το null. Θα το παραβλέψουμε όμως γιατί είναι ένα όνομα που δοθεί στον λογαριασμό αυτόν από τον [κάτοχό του](https://twitter.com/rul3r), οπότε δεν είναι ορθό να θεωρηθεί ως ελλιπής τιμή. Μέχρι την στιγμή της συγγραφής αυτού του notebook δεν βρέθηκε τρόπος να κάνουμε την isna() της pandas να παραβλέψει αυτό το χαρακτηριστικό, οπότε απλά θα το έχουμε υπόψη μας στη συνέχεια...

Οπότε ολοκληρώνεται εδώ ο σκοπός αυτού του notebook με την αποθήκευση του νέου DataFrame σε ένα αρχείο CSV.

In [ ]:
tweets.to_csv('Data/tweets/tweets_2021_with_users_matched.csv', index=False)